# Fault modeling example
Made for ESCAPE program - 2023



In [ ]:
# import statements - general statements
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.optimize

# import statements - specific to our problem.
import fault_model
import fault_plots

# this special Jupyter command makes zooming, panning and downloading the figures easy
#%matplotlib widget

# import statement for an interactive plot
from ipywidgets import interactive

%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

### Setting up a model

To create a model of surface displacement caused by slip on a fault, we use a fairly complicated analytic solution - check out the file "okada85.py" to see the details.


![okada fault model](https://www.researchgate.net/publication/229912692/figure/fig5/AS:271201062879250@1441670780804/Geometry-of-the-dislocation-source-model-modified-from-Okada-1985-The-model-is.png)


Because this is a complicated system, we'll hide the details inside "fault_model.py". To use that module, we run it as follows:

In [ ]:
# set up the parameters: first, define the upper left corner of the fault:
latcorner=35.5
loncorner=-117.3
depthcorner=0

# next, the strike and dip angles, and the size of the fault:
strike=-35
dip=85
L=100e3
W=20e3

# finally, the amounts of strike-slip and dip-slip:
ss=-100
ds=0

# now, we create the Fault "model" (this is just a python object that contains the fault information)
Fmod=fault_model.FaultModel()
Fmod.create_planar_model(latcorner=latcorner,loncorner=loncorner,depthcorner=depthcorner,strike=strike,dip=dip,L=L,W=W,nL=1,nW=1)

# define some random observation points:
obslat = 35.8 + np.random.randn(50)
obslon = -117.6 + np.random.randn(50)

# get the "G" matrix for our particular GPS site locations (lat,lon) - note this is Y,X in our notation, which can be confusing.
G=Fmod.get_greens(obslat,obslon)


In [ ]:
# what is the shape of G?
print(np.shape(G))

In [ ]:
# there are 3 times as many rows in G as data points! Why?
# this is because we need a separate line for each independent observation: for each point, we can measure the East, North, and Up displacements.


# let's use our model to compute displacements: Remember d=G*m. In this case, m is our slip
m=np.array([ss,ds])

# predict GPS displacements with d=G*m
gpsmotion = np.matmul(G,m)

# now extract the 3 components
predicted_E = gpsmotion[0::3] # this notation means start from first element (0), then take every 3rd one
predicted_N = gpsmotion[1::3]
predicted_U = gpsmotion[2::3]

# let's plot them
# plot the fault model and GPS
fig,ax=plt.subplots(1,1,figsize=(10,6.5))

# get the axes object so we can do normal things to it
ax.set_xlim([-119, -116])
ax.set_ylim([35, 37])
ax.grid()

up = plt.scatter(obslon, obslat,s=210,c=predicted_U,cmap='RdBu_r',marker='s')
plt.colorbar(up)
plt.clim([-5,5])
Fplot.plot_slip_outlines(Fmodel=Fmod)
#Fplot.plot_slip_patches(Fmodel=Fmod,slip=[ss],colorbar=False)

q=ax.quiver(obslon,obslat,predicted_E,predicted_N, scale=400,color='black')
ax.quiverkey(q,X=0.93,Y=0.95,U=20, label ='20 cm') # make quiver legend


In [ ]:
# define a function that implements the fault modeling functions defined in fault_model.py.
def create_fault_model(gpslon,gpslat,latcorner,loncorner,depthcorner,strike,dip,L,W,ss,ds):
    Fmod=fault_model.FaultModel()
    Fmod.create_planar_model(latcorner=latcorner,loncorner=loncorner,depthcorner=depthcorner,strike=strike,dip=dip,L=L,W=W,nL=1,nW=1)

    # get the "G" matrix for our particular GPS site locations (lat,lon) - note this is Y,X in our notation, which can be confusing.
    G=Fmod.get_greens(gpslat,gpslon)
    
    # m is the slip
    m=np.array([ss,ds])
    
    # predict GPS displacements with d=G*m
    gpsmotion = np.matmul(G,m)
    predicted_E = gpsmotion[0::3] # this notation means start from first element (0), then take every 3rd one
    predicted_N = gpsmotion[1::3]
    predicted_U = gpsmotion[2::3]
    
    return predicted_E,predicted_N,predicted_U,Fmod


# let's create a grid of data and play around with a fault model:

# units of latitude and longitude are in degrees. I'll create a "pretend" area around a point in the Mojave desert... you'll see why soon!
Npts=30
lon=np.linspace(-119,-116,Npts)
lat=np.linspace(34.5,37.5,Npts)

# create a "mesh grid" of locations
Lon,Lat = np.meshgrid(lon,lat)
Lon=np.reshape(Lon,Npts*Npts)
Lat=np.reshape(Lat,Npts*Npts)

# define a function for all the plotting and mapping commands, including running the fault model and comparing to data

# Note, the equals signs in the function definition set up a default value for each parameter
def plot_test_model(latcorner=35.5,loncorner=-117.3,depthcorner=0,strike=-35,dip=85,L=100e3,W=20e3,ss=-100,ds=0):
    # create a fault model
    # note, we pass gpslon and gpslat here, but it is not an input to this function. 
    # we are assuming it is already a global variable...
    predicted_E,predicted_N,predicted_U,Fmod = create_fault_model(Lon[:],Lat[:],latcorner,loncorner,depthcorner,strike,dip,L,W,ss,ds)

    # plot the fault model and GPS
    Fplot = fault_plots.FaultPlot2D(figsize=(10,6.5))
    
    # get the axes object so we can do normal things to it
    ax = Fplot.get_ax()
    ax.set_xlim([-119, -116])
    ax.set_ylim([35, 37])
    ax.grid()

    up = plt.scatter(Lon, Lat,s=210,c=predicted_U,cmap='RdBu_r',marker='s')
    plt.colorbar(up)
    plt.clim([-10,10])
    Fplot.plot_slip_outlines(Fmodel=Fmod)
    #Fplot.plot_slip_patches(Fmodel=Fmod,slip=[ss],colorbar=False)

    q=ax.quiver(Lon,Lat,predicted_E,predicted_N, scale=400,color='black')
    ax.quiverkey(q,X=0.93,Y=0.95,U=20, label ='20 cm') # make quiver legend
    
w = interactive(plot_test_model,latcorner=(35,37,.01),loncorner=(-118,-116.5,.01),depthcorner=(0,30e3),strike=(-180,180),dip=(0,180),L=(0,200e3),W=(0,200e3),ss=(-300,300),ds=(-100,100))
display(w)

### Fitting real data

That was great fun! Now let's try fitting some real GPS data:

This dataset contains the GPS displacements for the 100 stations nearest to the 2019 Ridgecrest, CA earthquake.

In [ ]:
# load GPS data
gpsdata = pd.read_csv('GPS_Ridgecrest.csv')

# I will convert them all to arrays at the start.
gpslon = np.asarray(gpsdata["X"])
gpslat = np.asarray(gpsdata["Y"])
gps_dE = np.asarray(gpsdata["E"])
gps_dN = np.asarray(gpsdata["N"])


In [ ]:
# create a map to view the GPS data

# sorry, this dataset did not include the vertical motions, only horizontal

fig = plt.figure(figsize = (10,8))
ax = fig.add_subplot(1, 1, 1,)
ax.set_xlim([-119.5, -116])
ax.set_ylim([34.5, 37])
ax.grid() # Add gridlines and labels

# Plot vectors
vecscale = 200 # note, larger scale means smaller arrows
q = ax.quiver(gpslon,gpslat,gps_dE,gps_dN, scale=vecscale)
plt.quiverkey(q,X=0.93,Y=0.95,U=20, label ='20 cm') # make quiver legend

plt.title('Ridgecrest earthquake data')
plt.show()

In [ ]:

# define a function for all the plotting and mapping commands, including running the fault model and comparing to data

# Note, the equals signs in the function definition set up a default value for each parameter
def plot_fault_and_gps(latcorner=35.5,loncorner=-117.3,depthcorner=0,strike=-35,dip=85,L=100e3,W=10e3,ss=-100,ds=0):
    # create a fault model
    # note, we pass gpslon and gpslat here, but it is not an input to this function. 
    # we are assuming it is already a global variable...
    predicted_E,predicted_N,predicted_U,Fmod = create_fault_model(gpslon,gpslat,latcorner,loncorner,depthcorner,strike,dip,L,W,ss,ds)

    # compute the misfit: sum of the residuals
    misfit=np.sum((predicted_E-gps_dE)**2+(predicted_N-gps_dN)**2)/(len(gps_dN)-1)
    
    # plot the fault model and GPS
    Fplot = fault_plots.FaultPlot2D(figsize=(10,7))
    Fplot.plot_slip_outlines(Fmodel=Fmod)

    # get the axes object so we can do normal things to it
    ax = Fplot.get_ax()
    ax.set_xlim([-119.5, -116])
    ax.set_ylim([34.5, 37])
    ax.set_title('Misfit: %f' %misfit)
    ax.grid()

    # commands for quiver are the same
    q = ax.quiver(gpslon,gpslat,gps_dE,gps_dN, scale=vecscale,color='blue')
    q2= ax.quiver(gpslon,gpslat,predicted_E,predicted_N, scale=vecscale,color='orange')
    ax.quiverkey(q,X=0.93,Y=0.95,U=20, label ='Data: 20 cm') # make quiver legend
    ax.quiverkey(q2,X=0.93,Y=0.85,U=20, label ='model') # make quiver legend

# why did we put all the plotting commands inside an annoying function? Because now we can make it interactive!
w = interactive(plot_fault_and_gps,latcorner=(35,37,.01),loncorner=(-118,-116.5,.01),depthcorner=(0,30e3),strike=(-180,180),dip=(0,180),L=(0,200e3),W=(0,20e3),ss=(-400,400),ds=(-200,200))
display(w)

In [ ]:
# let's use scipy.optimize.curve_fit to fit the GPS data.
# to use this, we need a function that takes as input the independent variables (station locations) as a single vector, and model parameters (fault),
# and outputs the dependent variables (gps displacements) as a single vector.

# note there is a numerical instability for this case. We fix the fault width to make things work better. Try changing it and see what happens!
fix_width = 10000 # 10km default

def fault_model_for_fitting(gps_locs,latcorner,loncorner,depthcorner,strike,dip,L,ss,ds):
    # for the modeling, we need the locations as x and y, not a single vector
    Ngps=int(np.size(gps_locs)/2)
    gpslat=gps_locs[:Ngps]
    gpslat=gps_locs[Ngps:]
    
    # create the model
    Fmod=fault_model.FaultModel()
    Fmod.create_planar_model(latcorner=latcorner,loncorner=loncorner,depthcorner=depthcorner,strike=strike,dip=dip,L=L,W=fix_width,nL=1,nW=1)
    
    # get the "G" matrix for our particular GPS site locations (lat,lon) - note this is Y,X in our notation, which can be confusing.
    G=Fmod.get_greens(gpslat,gpslon)
    
    # m is the slip
    m=np.array([ss,ds])
    
    # predict GPS displacements with d=G*m
    gpsmotion = np.matmul(G,m)
    predicted_E = gpsmotion[0::3] # this notation means start from first element (0), then take every 3rd one
    predicted_N = gpsmotion[1::3]
    
    # get the displacements as a single vector
    predicted_displacements=np.append(predicted_E,predicted_N)
    
    # return the predicted values
    return predicted_displacements

# create single vectors for the locations and displacements
gps_locations=np.append(gpslon,gpslat)
gps_displacements=np.append(gps_dE,gps_dN)

m,mcov = scipy.optimize.curve_fit(fault_model_for_fitting,gps_locations,gps_displacements,p0=[35.5,-117.3,0,-35,85,100e3,-100,0],maxfev=10000)
print(m)


In [ ]:

# get the predicted GPS displacements, by running the fault model again with the best fitting parameters (m)
gps_predicted=fault_model_for_fitting(gps_locations,m[0],m[1],m[2],m[3],m[4],m[5],m[6],m[7]) 

#split the displacements back into E and N
Ngps=int(np.size(gps_predicted)/2)
predicted_E=gps_predicted[:Ngps]
predicted_N=gps_predicted[Ngps:]

# compute the misfit: sum of the residuals
misfit=np.sum((predicted_E-gps_dE)**2+(predicted_N-gps_dN)**2)/(len(gps_dN)-1)

# create a plot with results
# fig = plt.figure(figsize = (7,7))
# ax = fig.add_subplot(1, 1, 1)

# create the best fitting model for plotting
Fmod=fault_model.FaultModel()
Fmod.create_planar_model(m[0],m[1],m[2],m[3],m[4],m[5],fix_width,nL=1,nW=1)

# plot the fault model and GPS
Fplot = fault_plots.FaultPlot2D(figsize=(10,7))
Fplot.plot_slip_outlines(Fmodel=Fmod)

# get the axes object so we can do normal things to it
ax = Fplot.get_ax()

# note that for a non-geographic plot, we use set_xlim and set_ylim instead of set_extent.
ax.set_xlim([-119, -115.5])
ax.set_ylim([34.5, 37])
ax.grid() # grid instead of gridlines


# commands for quiver are the same
q = ax.quiver(gpslon,gpslat,gps_dE,gps_dN, scale=vecscale,color='blue')
q2 = ax.quiver(gpslon,gpslat,predicted_E,predicted_N, scale=vecscale,color='orange')
ax.quiverkey(q,X=0.85,Y=0.85,U=20, label ='Dat: 20 cm') # make quiver legend
ax.quiverkey(q2,X=0.85,Y=0.75,U=20, label ='model') # make quiver legend
plt.title('Misfit: %f' %misfit)
print('Best fitting solution: lat %f, lon %f, depth %f, strike %f, dip %f, length %f, ss %f, ds %f' %(m[0],m[1],m[2],m[3],m[4],m[5],m[6],m[7]))


In [ ]:
# what is the magnitude?
length = m[5]
width = fix_width
slip = np.sqrt(m[6]**2+m[7]**2)/100 # units must be in meters, not cm

def get_magnitude(L,W,slip):
    return (2/3)*np.log10(1e7*30e9*L*W*slip) - 10.7

mag = get_magnitude(length,fix_width,slip)
print('Earthquake magnitude: %.2f' %mag)